# Sensor Computation Examples

This notebook has functions that are frequently used with sensor applications.

## Example: Subtract the mean from a stream

The output stream is the input stream after subtracting the mean of the input stream. The mean of the input stream is computed over a window whose size is specified as a parameter.

The output stream values are meaningful only after it has output window_size elements. Before that the computation is in start-up mode.

In [1]:
import os
import sys
sys.path.append("../")

import math
import numpy as np

# op, merge, source, sink are in agent_types
from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.op import map_window
from IoTPy.helper_functions.recent_values import recent_values

#----------------------------------------------------
# Agent that subtracts the mean in a stream
#----------------------------------------------------    
def subtract_mean(in_stream, out_stream, window_size):
    """
    Parameters
    ----------
    in_stream: Stream
       Stream of floats.
    out_stream: Stream
       Stream of floats
       The out_stream is the in_stream with the mean subtracted.
       So, the mean of the out_stream will be approximately 0.0.
    window_size: int
       positive value
       The size of the window on which the mean is calculated.
       The larger the window_size the more accurate is the subtraction
       of the mean, but also the slower the computation.

    """
    def f(window):
        """
        Parameters
        ----------
        window: list
           list of elements of a stream.
        """
        return window[-1] - np.mean(window)

    # Wrap function to create an agent.
    map_window(
        func=f, in_stream=in_stream, out_stream=out_stream,
        window_size=window_size, step_size=1)
    

In [2]:
def example_subtract_mean():
    x, y = Stream(), Stream()
    subtract_mean(in_stream=x, out_stream=y, window_size=4)
    
    # Execute a step
    x.extend(list(range(12)))
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

example_subtract_mean()

recent values of stream y are
[1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]


In [3]:
from IoTPy.agent_types.merge import zip_map

#----------------------------------------------------
# Agent that computes the magnitude of a vector
#----------------------------------------------------   
def magnitude_of_vector(in_streams, out_stream):
    """
    Parameters
    ----------
     in_streams: list of Stream
       Each component stream in the list is a stream of floats.
       The component streams are the streams of values of the
       coordinates of a vector. If there are input streams for
       x, y, z then the n-th element of the output stream is
       the mangitude of the vector [x[n], y[n], z[n]]
    out_stream: Stream
       Stream of floats
       The out_stream is the stream of magnitudes of the values.

    """
    def magnitude(coordinates):
        """
        Parameters
        ----------
        coordinates: list
           list of coordinates of a vector

        Returns
        -------
           float
           The magnitude of the vector

        """
        v = np.array(coordinates)
        return np.sqrt(v.dot(v))
    
    # Merge the input streams of each coordinate to create an output strema
    # of magnitudes.
    zip_map(func=magnitude, in_streams=in_streams, out_stream=out_stream)

In [4]:
def example_magnitude_of_vector():
    x, y= Stream(), Stream()
    magnitude_stream = Stream()
    magnitude_of_vector(in_streams=[x, y], out_stream=magnitude_stream)
    
    # Execute a step
    x.extend([3, 6, 9])
    y.extend([4, 8, 12])
    run()
    print ('recent values of magnitude_stream are')
    print (recent_values(magnitude_stream))

example_magnitude_of_vector()

recent values of magnitude_stream are
[5.0, 10.0, 15.0]
